### Import

In [20]:
import os
import random
import numpy as np
import pandas as pd
from tqdm import tqdm

import plotly.express as px

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import category_encoders as ce
import lightgbm as lgb

import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

In [21]:
# 동일한 결과 보장을 위해 Seed값을 고정합니다
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed를 42로 고정

In [22]:
# 제공된 train 데이터와 test 데이터를 불러옵니다
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [23]:
# 빠른 훈련을 위해 데이터 갯수 줄이기
# train = train.sample(100000)
# train.info()

### EDA 1 : Sparse and Dense

In [24]:
train.head()

,ID,Click,F01,F02,F03,F04,F05,F06,F07,F08,...,F30,F31,F32,F33,F34,F35,F36,F37,F38,F39
0,TRAIN_00000000,1,NSLHFNS,AVKQTCL,DTZFPRW,114.0,ISVXFVA,1,PQZBVMG,LPYPUNA,...,NZGEZLW,GTISJWW,380.0,2.0,AXQFZWC,IRUDRFB,NaN,TFJMLCZ,0.0,AURZYDY
1,TRAIN_00000001,0,VGIVWZQ,LSUSMVO,PQGWFJZ,26.0,NFRVLWS,43,IMPIGJT,MIGYEEG,...,NZGEZLW,GTISJWW,466.0,1.0,DRVVDHZ,IRUDRFB,19.0,AUGTURV,0.0,LUZRMLU
2,TRAIN_00000002,0,JCDXFYU,PILDDJU,IAGJDOH,119.0,LFPUEOV,0,FFUTIRZ,OFKQGTY,...,VHXETCF,KHZNEZF,197.0,0.0,QMOULXS,IRUDRFB,8.0,ZVSTLNM,0.0,MHBRSQK
3,TRAIN_00000003,1,PSMFWTP,ZYAVJHP,NaN,15.0,ATQPZSJ,26,ZDTZNSB,THBWWCD,...,IVIRTPR,GTISJWW,8640.0,0.0,IZLJUJS,IRUDRFB,14.0,ZBSRLCQ,0.0,GAZBSSZ
4,TRAIN_00000004,0,SLCRICD,QPQWGXA,NaN,13.0,CHZGJZR,20,PQZBVMG,MIGYEEG,...,NZGEZLW,WHSRKIM,41774.0,0.0,BHBIZCL,IRUDRFB,13.0,QHYLSBX,0.0,QTATWAY


### EDA 2 : Imbalance

In [25]:
click = train['Click'].value_counts(normalize=True)

click_figure = px.bar(click,
             x=['Not Clicked : 0', 'Clicked : 1'],
             y=click.values.tolist(),
             labels={'x': 'Value', 'y': 'Percentage'},
             width = 450,
             height = 500
            )

# 그래프 표시
click_figure.show()

### Data Preprocessing 1 : Select x, y

In [26]:
train_x = train.drop(columns=['ID', 'Click'])
train_y = train['Click']

test_x = test.drop(columns=['ID'])

### Data Preprocessing 2 : Fill NaN

In [27]:
for col in tqdm(train_x.columns):
    if train_x[col].isnull().sum() != 0:
        train_x[col].fillna(0, inplace=True)
        test_x[col].fillna(0, inplace=True)

100%|██████████| 39/39 [00:37<00:00,  1.05it/s]


### Data Preprocessing 3 : Count Encoding

In [28]:
encoding_target = list(train_x.dtypes[train_x.dtypes == "object"].index)

enc = ce.CountEncoder(cols = encoding_target).fit(train_x, train_y)
X_train_encoded = enc.transform(train_x)
X_test_encoded = enc.transform(test_x)

### Model Setting

In [30]:
def objectiveLGBM(trial: Trial, X, y):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
        'max_depth': trial.suggest_int('max_depth', 1, 20),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'learning_rate': trial.suggest_loguniform("learning_rate", 1e-8, 1e-2),
        "subsample": trial.suggest_float("subsample", 0.3, 1.0),
        # "subsample_freq": trial.suggest_int("subsample_freq", 1, 10),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'device':"gpu",
        'gpu_use_dp':True
    }
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
        
    model = lgb.LGBMClassifier(**param)
    model = model.fit(X_train, y_train)
    
    score = mean_squared_error(model.predict(X_test), y_test, squared=False)
    return score

In [31]:
study = optuna.create_study(direction='minimize')
study.optimize(lambda trial : objectiveLGBM(trial, X_train_encoded, train_y), n_trials=20)
print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))

optuna.visualization.plot_param_importances(study) # 파라미터 중요도 확인 그래프
optuna.visualization.plot_optimization_history(study) # 최적화 과정 시각화

[I 2024-05-26 18:39:16,578] A new study created in memory with name: no-name-7cd29153-27c5-46f9-9b74-d1920e1f795e


C:\Users\qwer\AppData\Local\Temp\ipykernel_5008\108474553.py:6: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.



[LightGBM] [Info] Number of positive: 5013320, number of negative: 20731531
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 6456
[LightGBM] [Info] Number of data points in the train set: 25744851, number of used features: 39
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 4070 Ti, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 16
[LightGBM] [Info] 35 dense feature groups (883.88 MB) transferred to GPU in 0.405041 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.194731 -> initscore=-1.419557
[LightGBM] [Info] Start training from score -1.419557


[I 2024-05-26 18:51:07,211] Trial 0 finished with value: 0.4410869199253874 and parameters: {'n_estimators': 1190, 'max_depth': 4, 'num_leaves': 184, 'learning_rate': 0.00015159740418497643, 'subsample': 0.6552358447870762, 'min_child_samples': 74}. Best is trial 0 with value: 0.4410869199253874.
C:\Users\qwer\AppData\Local\Temp\ipykernel_5008\108474553.py:6: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.



[LightGBM] [Info] Number of positive: 5012009, number of negative: 20732842
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 6446
[LightGBM] [Info] Number of data points in the train set: 25744851, number of used features: 39
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 4070 Ti, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 16
[LightGBM] [Info] 35 dense feature groups (883.88 MB) transferred to GPU in 0.423944 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.194680 -> initscore=-1.419882
[LightGBM] [Info] Start training from score -1.419882


...

[LightGBM] [Info] Number of positive: 5013154, number of negative: 20731697
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 6470
[LightGBM] [Info] Number of data points in the train set: 25744851, number of used features: 39
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 4070 Ti, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 16
[LightGBM] [Info] 35 dense feature groups (883.88 MB) transferred to GPU in 0.420083 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.194725 -> initscore=-1.419599
[LightGBM] [Info] Start training from score -1.419599


[I 2024-05-27 02:52:13,401] Trial 18 finished with value: 0.4411526968413599 and parameters: {'n_estimators': 1201, 'max_depth': 8, 'num_leaves': 75, 'learning_rate': 4.3326470701978e-05, 'subsample': 0.5721351803300325, 'min_child_samples': 100}. Best is trial 13 with value: 0.4226157117248266.
C:\Users\qwer\AppData\Local\Temp\ipykernel_5008\108474553.py:6: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.



[LightGBM] [Info] Number of positive: 5012964, number of negative: 20731887
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 6435
[LightGBM] [Info] Number of data points in the train set: 25744851, number of used features: 39
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 4070 Ti, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 16
[LightGBM] [Info] 35 dense feature groups (883.88 MB) transferred to GPU in 0.417220 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.194717 -> initscore=-1.419646
[LightGBM] [Info] Start training from score -1.419646


[I 2024-05-27 03:16:57,321] Trial 19 finished with value: 0.4238687045501581 and parameters: {'n_estimators': 1576, 'max_depth': 20, 'num_leaves': 227, 'learning_rate': 0.008780273761983077, 'subsample': 0.7172519358168575, 'min_child_samples': 73}. Best is trial 13 with value: 0.4226157117248266.


Best trial: score 0.4226157117248266,
params {'n_estimators': 2939, 'max_depth': 10, 'num_leaves': 256, 'learning_rate': 0.00947599327270648, 'subsample': 0.7989612028795253, 'min_child_samples': 99}


In [32]:
optuna.visualization.plot_param_importances(study)

In [33]:
model = lgb.LGBMClassifier(**study.best_trial.params)
model.fit(X_train_encoded, train_y)

[LightGBM] [Info] Number of positive: 5569860, number of negative: 23035531
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.442915 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6454
[LightGBM] [Info] Number of data points in the train set: 28605391, number of used features: 39
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.194714 -> initscore=-1.419668
[LightGBM] [Info] Start training from score -1.419668
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

LGBMClassifier(learning_rate=0.00947599327270648, max_depth=10,
               min_child_samples=99, n_estimators=2939, num_leaves=256,
               subsample=0.7989612028795253)

### Model Train and Inference

In [34]:
pred = model.predict_proba(X_test_encoded)
display(model.classes_)
display(pred)

array([0, 1], dtype=int64)

array([[0.78186456, 0.21813544],
       [0.87413343, 0.12586657],
       [0.88083507, 0.11916493],
       ...,
       [0.9210273 , 0.0789727 ],
       [0.80686313, 0.19313687],
       [0.72118189, 0.27881811]])

### Submission

In [35]:
sample_submission = pd.read_csv('sample_submission.csv')
sample_submission

,ID,Click
0,TEST_0000000,0
1,TEST_0000001,0
2,TEST_0000002,0
3,TEST_0000003,0
4,TEST_0000004,0
...,...,...
4538536,TEST_4538536,0
4538537,TEST_4538537,0
4538538,TEST_4538538,0
4538539,TEST_4538539,0


In [36]:
sample_submission['Click'] = pred[:,1]
sample_submission

,ID,Click
0,TEST_0000000,0.218135
1,TEST_0000001,0.125867
2,TEST_0000002,0.119165
3,TEST_0000003,0.407183
4,TEST_0000004,0.451049
...,...,...
4538536,TEST_4538536,0.220562
4538537,TEST_4538537,0.289496
4538538,TEST_4538538,0.078973
4538539,TEST_4538539,0.193137


In [37]:
sample_submission.to_csv('baseline_submission.csv', index=False)